# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [1]:
target: str = ''
start: str = '2023-01-01'
end: str = '2025-06-10'
fixed_date: str = 'true'

## Setup

Import some libraries

In [2]:
import json
from datetime import date
from ast import literal_eval

import petl as etl
from utils.paths import PUBLISHED, SITE
from utils.themes.programme import Programme, ProgrammeSlice

Set the target directory, and ensure it exists

In [3]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [4]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end),
    date.today(),
)

Write the timestamp file

In [5]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end', 'updated'], (str(x) for x in date_range))), f)

Load programme data

In [6]:
programme_data = ProgrammeSlice(range=date_range[0:2])

## Events & projects data

In [7]:
events = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'project_id', 'evaluation_category'], sum, 'events', field='events')
    
    # TODO fix this kludge
    .groupselectfirst('project_name', 'events')
)

In [8]:
project_summary = {
    'total': events.distinct('project_name').nrows(),
    'by_category': dict(events.aggregate('evaluation_category', len).records()),
    'names': list(events.cut('project_name').distinct().values('project_name')),
}

In [9]:
events_summary = {
    'total': sum(events.values('events')),
    'by_category': dict(events.aggregate('evaluation_category', sum, 'events').records()),
    'by_project': {
        r[0]: {'category': r[2], 'count': r[3]}
        for r
        in events.records()
    },
}

In [10]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        events_summary,
        f,
        indent=2
    )

with open(TARGET / 'projects.json', 'w') as f:
    json.dump(
        project_summary,
        f,
        indent=2
    )

## Audiences

In [11]:
ticketed_events = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
    .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
    .selectrangeopen('start', date_range[0], date_range[1])
    .selectcontains('eventType', 'MainProgrammeEvent')
)

In [12]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(ticketed_events.cut('instance_id', 'event_name'))
    .cut('event_name', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
)

In [13]:
tickets_scanned = dict(
    etl
    .fromcsv(PUBLISHED / 'manual/spektrix-sales-and-scans.csv')
    .convert('Event Date', etl.dateparser('%Y-%m-%d'))
    .selectrangeopen('Event Date', date_range[0], date_range[1])
    .convertnumbers()
    .aggregate(None, {
        'sold': ('Sold', sum),
        'scanned': ('Scanned', sum)
    })
    .melt(variables=['sold', 'scanned'])
    .records()
)

In [14]:
ticketed_audience = {
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
        'by_project': dict(tickets_sold.aggregate('event_name', sum, 'count_of_tickets', field='count_of_tickets').records()),
        'by_geo': dict(tickets_sold.aggregate('geography_code', sum, 'count_of_tickets', field='count_of_tickets').records()),
    },
    'scanned': {
        'total': tickets_scanned
    }
}

In [15]:
audience_data = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'project_id', 'evaluation_category'], sum, 'audience', field='audience')

    # TODO fix this kludge
    .groupselectfirst('project_name', 'audience')
)


In [16]:
audience = {
    'total': sum(audience_data.values('audience')),
    'by_category': dict(
        audience_data
        .aggregate(['evaluation_category'], sum, 'audience')
        .records()
    ),
    'by_project': dict(
        audience_data
        .aggregate(['project_name'], sum, 'audience')
        .records()
    ),
}

In [17]:
with open(TARGET / 'audience.json', 'w') as f:
    json.dump(
        audience | {
            'ticketed': ticketed_audience,
            'unticketed': None,
        },
        f,
        indent=2
    )

## Participants

In [18]:
manual_participants = (
    etl
    .fromcsv(PUBLISHED / 'manual/manual-participants.csv')

    .convert('Date', lambda d: date.fromisoformat(d))
    .selectge('Date', date_range[0])
    .selectle('Date', date_range[1])
    .selectnotin('variable', ['Schools engaged'])
    .convertnumbers()
)

In [19]:
schools_engaged = sum(
    etl
    .fromcsv(PUBLISHED / 'manual/manual-participants.csv')

    .convert('Date', lambda d: date.fromisoformat(d))
    .selectge('Date', date_range[0])
    .selectle('Date', date_range[1])
    .selectin('variable', ['Schools engaged'])
    .convertnumbers()
    .values('value')
)

In [20]:
participant_summary = {
    'total': sum(manual_participants.values('value')),
    'by_partipant_type': dict(manual_participants.aggregate('variable', sum, 'value').records()),
    # TODO make this nested by variable
    'by_partipant_type_and_project': [i for i in manual_participants.aggregate(['variable', 'Project'], sum, 'value').dicts()],
    'creative_learning': {
        'schools_engaged': schools_engaged
    }
}

## Volunteers

In [21]:
signups = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/checkpoint-updates.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selectle('date', date_range[1])
    .convertnumbers(strict=False)
)

In [22]:
shifts = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selecteq('type', 'BD25 Event')
    .selectrangeopen('date', date_range[0], date_range[1])
    .convertnumbers(strict=False)
)

In [23]:
volunteer_summary = {
    'events': {
        'count': shifts.cut('rosterfy_event_name').distinct().nrows(),
        'names': list(shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    },
    'shifts': {
        'count': shifts.nrows(),
        'attended': sum(shifts.values('attended')),
        'hours': sum(shifts.values('hours')),
    },
    'people': {
        'checkpoints': dict(signups.aggregate('checkpoint', sum, 'count').records()),
    },
}

In [24]:
with open(TARGET / 'participants.json', 'w') as f:
    json.dump(
        {
            'total': participant_summary['total'] +
                volunteer_summary['people']['checkpoints']['1. Monitoring & Evaluation'] +
                volunteer_summary['people']['checkpoints']['2. Sign Up to Induction'] +
                volunteer_summary['people']['checkpoints']['3. Fully Inducted Volunteers'],
            'participants': participant_summary,
            'volunteers': volunteer_summary,
        },
        f,
        indent=2
    )